In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00


In [17]:
import joblib
import pandas as pd
import gradio as gr
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

nltk.download('stopwords')

import re
import unicodedata
import spacy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
from google.colab import files


uploaded = files.upload()


Saving vectorizer.pkl to vectorizer.pkl


In [14]:
model = joblib.load('review_classifier_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

In [38]:
#Pré processamento

def preprocess_text(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  text = text.lower()

  #[^a-zA-Z0-9\s]
  text = re.sub(r'[^\w\s]', '', text)

  stop_words = set(stopwords.words('english'))
  text = ' '.join([word for word in text.split() if word not in stop_words])

  #Lemmatizing
  nlp = spacy.load('en_core_web_sm')
  doc = nlp(text)
  text = ' '.join([token.lemma_ for token in doc])

  return text


# Função de classificação
def classify_sentiment(review_text):

    processed_text = preprocess_text(review_text)

    text_tfidf = vectorizer.transform([processed_text])

    pred = model.predict_proba(text_tfidf)[0]

    return {'Positive': float(pred[1]),  #Probabilidade de ser positivo
            'Negative': float(pred[0])}  #Probabilidade de ser negativo

# Interface Gradio
iface = gr.Interface(
    fn=classify_sentiment,
    inputs=gr.Textbox(label="Type your review", lines=3),
    outputs="label",
    title="Sentiment Analysis on a product/food/movie review",
    description="Type a review in english, and the model will classify as positive or negative.",
    examples=[
        ["This movie was amazing! The acting was superb."],
        ["Terrible taste and bad presentation. Would not recommend."],
        ["It was okay, nothing special."]
    ],
    theme="compact",
    #interpretation="default"
)

iface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1115: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-67e3022f-2b83117e3b20984b2f3c9499;fade397c-ec98-465d-8417-665673c478cd)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ca0e14a88498f31d44.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Teste fora do gradle

In [37]:
test_text = "One of the most horrible movies i watched"
processed = preprocess_text(test_text)
vectorized = vectorizer.transform([processed])
print(model.predict_proba(vectorized))

[[0.66536327 0.33463673]]
